In [4]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
# from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_provider import IBMProvider
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit.circuit.random import random_circuit
from collections import defaultdict
import numpy as np

In [6]:
service = IBMProvider(token="#### PLACE TOKEN HERE ####")
sim_ideal = AerSimulator()
backend = service.get_backend("ibm_brisbane")
noise_model = NoiseModel.from_backend(backend)
sim_noise = AerSimulator(noise_model=noise_model)

In [5]:
def execute_circuit(qc,ideal_run=True,noisy_run=True):
    if ideal_run and not noisy_run:
        passmanager = generate_preset_pass_manager(optimization_level=0, backend=sim_noise)
        circ_tnoise = passmanager.run(qc)
        result = sim_ideal.run(circ_tnoise,run_options={"shots":1024}).result()
        ideal_counts = result.get_counts()
        return ideal_counts
    
    elif not ideal_run and noisy_run:
        passmanager = generate_preset_pass_manager(optimization_level=0, backend=sim_noise)
        circ_tnoise = passmanager.run(qc)
        noise_result = sim_noise.run(circ_tnoise,run_options={"shots":1024}).result()
        noisy_counts = noise_result.get_counts()
        return noisy_counts
    
    elif ideal_run and noisy_run:
        passmanager = generate_preset_pass_manager(optimization_level=0, backend=sim_noise)
        circ_tnoise = passmanager.run(qc)
        noise_result = sim_noise.run(circ_tnoise,run_options={"shots":1024}).result()
        result = sim_ideal.run(circ_tnoise,run_options={"shots":1024}).result()
        noisy_counts = noise_result.get_counts()
        ideal_counts = result.get_counts()
        return {"ideal":ideal_counts,"noisy":noisy_counts}
    else:
        raise("Not Implemented")
        

def calculate_features(noisy_count,ideal_count=None):
    if isinstance(noisy_count,list):
        
        total_noisy_runs = defaultdict(list)
        for d in noisy_count:
            for key, value in d.items():
                total_noisy_runs[key].append(value/sum(d.values()))
        
        feature_result = {}
        
        for each_state in total_noisy_runs:
            median_prob = np.round(np.median(total_noisy_runs[each_state]),3)
            odds = [v/(1-v) for v in total_noisy_runs[each_state]]
            median_odds = np.round(np.median(odds),3)
            probf = [1-v for v in total_noisy_runs[each_state]]
            median_probf = np.round(np.median(probf),3)
            feature_result[each_state] = {"POS":median_prob,"ODR":median_odds,"POF":median_probf}


        if ideal_count!=None:
            total_ideal_runs = defaultdict(list)
            for d in ideal_count:
                for key, value in d.items():
                    total_ideal_runs[key].append(value/sum(d.values()))

            for each_state in feature_result:
                if each_state in total_ideal_runs.keys():
                    feature_result[each_state]["ideal_prob"] = np.round(np.median(total_ideal_runs[each_state]),3)
                else:
                    feature_result[each_state]["ideal_prob"] = -1
                
        return feature_result

    else:
        raise("Unknow type for counts")